<a href="https://colab.research.google.com/github/ApurbaPaul-NLP/Machine-Learning/blob/main/22_08_2022_Introduction_to_modules%2C_layers%2C_and_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

A model is, abstractly A function that computes something on tensors.

Here in a Model Some variables can be updated in response to training.

In [1]:
import tensorflow as tf
import numpy as np
# Load the TensorBoard notebook extension
%load_ext tensorboard

In TensorFlow, most high-level implementations of layers and models, such as Keras or Sonnet, are built on the same foundational class: tf.Module.

In [2]:
class SimpleModule(tf.Module):
  def __init__(self, name=None):
    super().__init__(name=name)
    self.a_variable = tf.Variable(5.0, name="train_me")
    self.non_trainable_variable = tf.Variable(5.0, trainable=False, name="do_not_train_me")
  def __call__(self, x):
    return self.a_variable * x + self.non_trainable_variable

simple_module = SimpleModule(name="simple")

simple_module(tf.constant(5.0))

<tf.Tensor: shape=(), dtype=float32, numpy=30.0>

In [8]:
# All trainable variables
print("trainable variables:", simple_module.trainable_variables)
# Every variable
print("all variables:", simple_module.variables)

trainable variables: (<tf.Variable 'train_me:0' shape=() dtype=float32, numpy=5.0>,)
all variables: (<tf.Variable 'train_me:0' shape=() dtype=float32, numpy=5.0>, <tf.Variable 'do_not_train_me:0' shape=() dtype=float32, numpy=5.0>)


# **What is a "callable"?**

In [3]:
class Foo:
  def __call__(self):
    print ('called')

foo_instance = Foo()
foo_instance()

called


    Consider this foo() as foo.__call__() in the above example.

    Where foo can be any object that responds to __call__. 

When I say any object, I mean it: built-in types, your own classes and their instances.

In the case of built-in types, when you write:

      int('10')

      unicode(10)

You're essentially doing:

        int.__call__('10')

        unicode.__call__(10)

You're essentially doing 

      type(int).__call__(int, '10') and 

      type(unicode).__call__(unicode, '10')



    __call__ makes any object be callable as a function.

    In Python a callable is an object which type has a __call__ method.

To check function or method of class is callable or not that means we can call that function.
    



In [4]:
class Adder(object):
  def __init__(self, val):
    self.val = val

  def __call__(self, val):
    return self.val + val

func = Adder(5)
print (func(3))

8


In [6]:
class A:
    def __init__(self,val):
        self.val = val
    def bar(self):
        print ("bar")

obj = A(8)      
callable(obj.bar)

True

# **Here is an example of a two-layer linear layer model made out of modules.**

In [10]:
#First a dense (linear) layer:
class Dense(tf.Module):
  def __init__(self,in_features,out_features,name=None):
    super().__init__(name=name)
    self.w=tf.Variable(tf.random.normal([in_features,out_features]),name='w')
    self.b=tf.Variable(tf.zeros([out_features]),name='b')

  def __call__(self,x):
    y=tf.matmul(x,self.w)+self.b
    return tf.nn.relu(y)  

In [18]:
# The complete model, which makes two layer instances and applies them:
class SimpleModule(tf.Module):
  def __init__(self,name=None):
    super().__init__(name=name)

    self.dense1=Dense(in_features=3,out_features=3)  
    self.dense2=Dense(in_features=3,out_features=2)

  def __call__(self,x):
    x=self.dense1(x)
    return self.dense2(x)  

mymodule=SimpleModule(name="the_Model") 
print(mymodule(tf.constant([[2.,2.,2.]])))   

tf.Tensor([[0. 0.]], shape=(1, 2), dtype=float32)


In [19]:
print("Submodules:", mymodule.submodules)

Submodules: (<__main__.Dense object at 0x7fbbf5f9ad50>, <__main__.Dense object at 0x7fbbf5f9a110>)


In [20]:
for var in mymodule.variables:
  print(var, "\n")

<tf.Variable 'b:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)> 

<tf.Variable 'w:0' shape=(3, 3) dtype=float32, numpy=
array([[ 0.20196287,  1.3003284 ,  1.257852  ],
       [-0.7809273 ,  0.2719014 , -1.395672  ],
       [ 0.8206721 , -2.3030934 ,  0.52513075]], dtype=float32)> 

<tf.Variable 'b:0' shape=(2,) dtype=float32, numpy=array([0., 0.], dtype=float32)> 

<tf.Variable 'w:0' shape=(3, 2) dtype=float32, numpy=
array([[ 0.72945565,  0.93910336],
       [ 1.3968815 ,  1.1923939 ],
       [-0.7697298 , -0.74939656]], dtype=float32)> 

